In [ ]:
!pip install transformers torch
 

In [ ]:
import torch
from torch.utils.data import DataLoader
from ArticleDataset import ArticlesDataset
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import os
from tqdm import tqdm
from transformers import AdamW
import numpy as np
import pyarrow.parquet as pq

import os
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'


tokenizer = AutoTokenizer.from_pretrained("Maltehb/danish-bert-botxo")
model = AutoModel.from_pretrained("Maltehb/danish-bert-botxo")


dataset_type = "ebnerd_demo"
base_path = os.path.join(".", dataset_type)
text_column = 'body'
articles_columns = ["article_id", "premium", "category", "sentiment_score", "body"]
#load data articles
articles_path = os.path.join(base_path, "articles.parquet")

parquet_file = pq.ParquetFile(articles_path)
print(parquet_file.schema.names)

articles = pd.read_parquet(articles_path)[articles_columns]


num_epochs = 10
articles_dataset = ArticlesDataset(articles, tokenizer)
articles_loader = DataLoader(articles_dataset, batch_size=16, shuffle=True)


# Load model in evaluation mode
model.eval()
article_ids = []
all_embeddings = []

# Extract embeddings
with torch.no_grad():
    for batch in tqdm(articles_loader):
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Get the token embeddings
        all_embeddings.append(embeddings)
        article_ids.extend(batch['article_id'].numpy())

# Concatenate all embeddings into one array
all_embeddings = np.concatenate(all_embeddings, axis=0)

# Create a df to store article IDs with their embeddings
embeddings_df = pd.DataFrame(all_embeddings)
embeddings_df['article_id'] = article_ids

# Save embeddings to a file for later use
embeddings_df.to_parquet('./article_embeddings.parquet')

print("Embeddings saved successfully.")


#touseembeddingslater
embeddings_df = pd.read_parquet('./article_embeddings.parquet')
# Use embeddings_df as needed

['article_id', 'title', 'subtitle', 'last_modified_time', 'premium', 'body', 'published_time', 'item', 'article_type', 'url', 'item', 'item', 'item', 'category', 'item', 'category_str', 'total_inviews', 'total_pageviews', 'total_read_time', 'sentiment_score', 'sentiment_label']


  3%|▎         | 23/737 [05:59<3:21:03, 16.90s/it]